In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_moons
import matplotlib.animation as animation
from IPython.display import HTML
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
np.random.seed(2)

In [3]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from scipy.interpolate import CubicSpline, PchipInterpolator
import matplotlib.pyplot as plt

class GradientSMPA(BaseEstimator, ClassifierMixin):
    def __init__(self, learning_rate=0.05, epochs=100, random_state=7, verbose=False,
                 lambda_reg=0.0001, patience=10, decay_factor=0.9, min_learning_rate=1e-6,
                 n_control_points=6, smoothing_factor=0.0001, spline_type='cubic', epsilon=1e-4,
                 track_history=False):
        self.learning_rate = learning_rate
        self.initial_learning_rate = learning_rate
        self.epochs = epochs
        self.random_state = random_state
        self.verbose = verbose
        self.lambda_reg = lambda_reg
        self.patience = patience
        self.decay_factor = decay_factor
        self.min_learning_rate = min_learning_rate
        self.n_control_points = n_control_points
        self.smoothing_factor = smoothing_factor
        self.spline_type = spline_type
        self.epsilon = epsilon
        self.track_history = track_history  # New flag to toggle history tracking
        if spline_type not in ['cubic', 'pchip']:
            raise ValueError("spline_type must be 'cubic' or 'pchip'")
        np.random.seed(random_state)

    def _calculate_class_means(self, X, y):
        mask_1 = y == 1
        self.m1 = np.mean(X[mask_1], axis=0)
        self.m0 = np.mean(X[~mask_1], axis=0)

    def _initialize_control_points(self, X):
        x_min, x_max = X[:, 0].min(), X[:, 0].max()
        y_min, y_max = X[:, 1].min(), X[:, 1].max()
        y_range = y_max - y_min
        self.control_x = np.linspace(x_min, x_max, self.n_control_points)
        y_mid = (self.m0[1] + self.m1[1]) / 2
        self.control_y = np.random.uniform(y_mid - y_range * 0.05, y_mid + y_range * 0.05, self.n_control_points)
        if self.n_control_points > 2:
            self.control_y[0] = self.control_y[1]
            self.control_y[-1] = self.control_y[-2]
        self.initial_control_x = self.control_x.copy()
        self.initial_control_y = self.control_y.copy()

    def _fit_spline(self):
        if self.spline_type == 'cubic':
            self.spline = CubicSpline(self.control_x, self.control_y, bc_type='clamped')
        else:
            self.spline = PchipInterpolator(self.control_x, self.control_y)

    def _calculate_displacement(self, X):
        return X[:, 1] - self.spline(X[:, 0])

    def _update_pseudo_labels(self, X, y):
        m1_displacement = self._calculate_displacement(self.m1.reshape(1, -1))[0]
        self.class_1_pseudo = 1 if m1_displacement > 0 else -1
        self.class_0_pseudo = -self.class_1_pseudo
        return np.where(y == 1, self.class_1_pseudo, self.class_0_pseudo)

    def _compute_gradients(self, X, y):
        displacements = self._calculate_displacement(X)
        pseudo_labels = self._update_pseudo_labels(X, y)
        errors = displacements * pseudo_labels <= 0
        error_indices = np.where(errors)[0]

        if len(error_indices) == 0:
            return np.zeros_like(self.control_x), np.zeros_like(self.control_y)

        X_err = X[error_indices, 0]
        y_err = y[error_indices]
        ti = np.where(y_err == 1, 1, -1)
        sign_theta = self.class_1_pseudo

        base_spline = self.spline(X_err)
        n_points = len(self.control_y)
        grad_y = np.zeros(n_points)
        perturbed_diff = np.zeros((n_points, len(X_err)))

        if self.spline_type == 'cubic':
            for j in range(n_points):
                control_y_pert = self.control_y.copy()
                control_y_pert[j] += self.epsilon
                spline_pert = CubicSpline(self.control_x, control_y_pert, bc_type='clamped')
                perturbed_diff[j] = spline_pert(X_err) - base_spline
        else:  # PCHIP
            for j in range(n_points):
                control_y_pert = self.control_y.copy()
                control_y_pert[j] += self.epsilon
                spline_pert = PchipInterpolator(self.control_x, control_y_pert)
                perturbed_diff[j] = spline_pert(X_err) - base_spline

        grad_y += (ti * sign_theta) @ (perturbed_diff.T / self.epsilon)
        grad_y[1:] += 2 * self.lambda_reg * (self.control_y[1:] - self.control_y[:-1])
        grad_y[:-1] -= 2 * self.lambda_reg * (self.control_y[1:] - self.control_y[:-1])

        grad_x = np.zeros_like(self.control_x)
        grad_x[1:] += 2 * self.lambda_reg * (self.control_x[1:] - self.control_x[:-1])
        grad_x[:-1] -= 2 * self.lambda_reg * (self.control_x[1:] - self.control_x[:-1])

        return grad_x, grad_y

    def fit(self, X, y):
        if not set(np.unique(y)).issubset({0, 1}):
            raise ValueError("Labels must be 0 and 1")
        if X.shape[1] != 2:
            raise ValueError("2D-only algorithm for now!")

        X = np.asarray(X)
        y = np.asarray(y)
        self._calculate_class_means(X, y)
        self._initialize_control_points(X)
        self._fit_spline()

        current_lr = self.initial_learning_rate
        best_error = float('inf')
        best_control_x = self.control_x.copy()
        best_control_y = self.control_y.copy()
        patience_counter = 0

        if self.track_history:
            self.error_history_ = []
            self.control_point_history = [(self.control_x.copy(), self.control_y.copy())]

        for epoch in range(self.epochs):
            pseudo_labels = self._update_pseudo_labels(X, y)
            displacements = self._calculate_displacement(X)
            error_count = np.sum(displacements * pseudo_labels <= 0)

            if self.verbose and epoch % 5 == 0:
                print(f"Epoch {epoch}: Errors = {error_count}, LR = {current_lr:.6f}")

            if error_count < best_error:
                best_error = error_count
                best_control_x = self.control_x.copy()
                best_control_y = self.control_y.copy()
                self.class_1_pseudo_best = self.class_1_pseudo
                patience_counter = 0
                self.best_epoch = epoch
            else:
                patience_counter += 1
                if patience_counter >= self.patience:
                    current_lr = max(current_lr * self.decay_factor, self.min_learning_rate)
                    patience_counter = 0
                    if current_lr == self.min_learning_rate:
                        if self.verbose:
                            print(f"Min learning rate reached at epoch {epoch}")
                        break

            if self.track_history:
                self.error_history_.append(error_count)
                self.control_point_history.append((self.control_x.copy(), self.control_y.copy()))

            grad_x, grad_y = self._compute_gradients(X, y)
            self.control_x -= current_lr * grad_x
            self.control_y -= current_lr * grad_y
            self._fit_spline()

        self.control_x = best_control_x
        self.control_y = best_control_y
        self.class_1_pseudo = self.class_1_pseudo_best
        self._fit_spline()
        return self

    def predict(self, X):
        displacements = self._calculate_displacement(X)
        return np.where(displacements > 0,
                        1 if self.class_1_pseudo > 0 else 0,
                        0 if self.class_1_pseudo > 0 else 1)

    def predict_proba(self, X):
        displacements = self._calculate_displacement(X)
        raw_probs = 1 / (1 + np.exp(-displacements * self.class_1_pseudo * 0.5))
        return np.column_stack([1 - raw_probs, raw_probs]) if self.class_1_pseudo > 0 else np.column_stack([raw_probs, 1 - raw_probs])

    def plot_convergence(self, figsize=(10, 4)):
        if not self.track_history or not hasattr(self, 'error_history_'):
            print("Convergence plotting requires track_history=True and a fitted model.")
            return None
        fig, ax = plt.subplots(figsize=figsize)
        ax.plot(self.error_history_, 'b-', label='Errors')
        ax.set_title('Error Convergence')
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Number of Errors')
        ax.grid(True)
        ax.legend()
        plt.tight_layout()
        return fig

    def plot_boundary(self, X, y, figsize=(8, 6)):
        fig = plt.figure(figsize=figsize)
        plt.scatter(X[:, 0], X[:, 1], c=y, cmap='bwr', alpha=0.5)
        x_range = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)

        if self.spline_type == 'cubic':
            initial_spline = CubicSpline(self.initial_control_x, self.initial_control_y, bc_type='clamped')
        else:
            initial_spline = PchipInterpolator(self.initial_control_x, self.initial_control_y)
        y_initial = initial_spline(x_range)
        plt.plot(x_range, y_initial, 'r--', label='Initial Boundary', alpha=0.7)

        y_final = self.spline(x_range)
        plt.plot(x_range, y_final, 'g-', label='Final Boundary')

        plt.scatter(self.initial_control_x, self.initial_control_y, c='orange', marker='o',
                    label='Initial Control Points', alpha=0.7)
        plt.scatter(self.control_x, self.control_y, c='k', marker='x',
                    label='Final Control Points')

        plt.legend()
        plt.title('GradientSMPA Decision Boundary: Initial vs Final')
        plt.xlabel('Feature 1')
        plt.ylabel('Feature 2')
        return fig

In [4]:
df = pd.read_csv("haberman.csv", header = None)
df.rename(columns={3:'label'}, inplace = True)
df.replace({'label' : {1 : 1, 2 : 0}}, inplace=True)
X = df.iloc[:, :-1:2]
y = df.iloc[:, -1]

In [5]:
df.columns

Index([0, 1, 2, 'label'], dtype='object')

In [6]:
X.head()

,0,2
0,30,1
1,30,3
2,30,0
3,31,2
4,31,4


In [7]:
print(y)

0      1
1      1
2      1
3      1
4      1
      ..
301    1
302    1
303    1
304    0
305    0
Name: label, Length: 306, dtype: int64


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12)

In [9]:
scaler = MinMaxScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

array([[0.41509434, 0.        ],
       [0.71698113, 0.        ],
       [0.54716981, 0.        ],
       [0.54716981, 0.        ],
       [0.49056604, 0.        ],
       [0.73584906, 0.        ],
       [0.35849057, 0.        ],
       [0.37735849, 0.11538462],
       [0.        , 0.05769231],
       [0.66037736, 0.        ],
       [0.13207547, 0.11538462],
       [0.83018868, 0.        ],
       [0.52830189, 0.01923077],
       [0.41509434, 0.07692308],
       [0.58490566, 0.        ],
       [0.45283019, 0.44230769],
       [0.28301887, 0.        ],
       [0.16981132, 0.        ],
       [0.24528302, 0.03846154],
       [0.35849057, 0.        ],
       [0.66037736, 0.        ],
       [0.54716981, 0.01923077],
       [0.50943396, 0.        ],
       [0.69811321, 0.01923077],
       [0.33962264, 0.03846154],
       [0.83018868, 0.05769231],
       [0.41509434, 0.01923077],
       [0.28301887, 0.26923077],
       [0.0754717 , 0.13461538],
       [0.26415094, 0.        ],
       [0.

In [10]:
smpa = GradientSMPA()

param_grid = {
    'learning_rate': [0.001, 0.005, 0.01],  # Standard tuning range for LR
    'epochs': [300],  # Allowing both standard and longer training
    'decay_factor': [0.99, 0.999],  # Testing different decay behaviors
    'min_learning_rate': [1e-6],  # Ensuring it doesn't decay too much
    'n_control_points': [10, 15, 20],  # Key for spline flexibility
    'smoothing_factor': [0.0, 0.000001, 0.0001],  # Testing impact of smoothing
    'spline_type': ['pchip', 'cubic'],  # Different spline formulations
    'lambda_reg': [0.0, 0.001, 0.01],  # Regularization strength
    'patience': [5, 10, 20],  # Controlling early stopping
    'epsilon': [1e-4, 1e-3, 1e-2],  # Convergence tolerance
}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=3)
grid_search = GridSearchCV(smpa, param_grid=param_grid, n_jobs = -1, scoring = 'accuracy', cv = cv, verbose=3)

grid_search.fit(X_train, y_train)

clf = grid_search.best_estimator_

y_pred = clf.predict(X_test)

Fitting 3 folds for each of 2916 candidates, totalling 8748 fits
[CV 1/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=0.0, spline_type=pchip;, score=nan total time=   2.4s


/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: (slice(None, None, None), 1)

During handling of the above exception, another exception occurr

[CV 2/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=1e-06, spline_type=pchip;, score=nan total time=   2.2s


/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: (slice(None, None, None), 1)

During handling of the above exception, another exception occurr

[CV 2/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=0.0, spline_type=cubic;, score=nan total time=   1.6s
[CV 2/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=1e-06, spline_type=cubic;, score=nan total time=   1.8s
[CV 1/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=0.0, spline_type=cubic;, score=nan total time=   1.8s
[CV 3/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=0.0, spline_type=pchip;, score=nan total time=   1.8s
[CV 2/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.00

...

/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: (slice(None, None, None), 1)

During handling of the above exception, another exception occurr

[CV 2/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=0.0001, spline_type=pchip;, score=nan total time=   2.0s
[CV 3/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=1e-06, spline_type=pchip;, score=nan total time=   2.3s
[CV 1/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=0.0001, spline_type=pchip;, score=nan total time=   2.4s


/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/soulangel443/.local/share/mise/installs/python/latest/lib/python3.12/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: (slice(None, None, None), 1)

During handling of the above exception, another exception occurr

[CV 3/3] END decay_factor=0.99, epochs=300, epsilon=0.0001, lambda_reg=0.0, learning_rate=0.001, min_learning_rate=1e-06, n_control_points=10, patience=5, smoothing_factor=0.0001, spline_type=cubic;, score=nan total time=   1.8s


KeyboardInterrupt: 

In [ ]:
y_pred = clf.predict(X_test)
print("Best score = ", grid_search.best_score_)
print(classification_report(y_pred, y_test))


In [12]:
df = pd.read_csv("haberman.csv", header = None)
df.rename(columns={3:'label'}, inplace = True)
df.replace({'label' : {1 : 1, 2 : 0}}, inplace=True)
X = df.iloc[:, :-1:2]
y = df.iloc[:, -1]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 12)

In [ ]:
svm = SVC()

svm_grid =  {
                'C': [0.1, 1, 10, 50, 100],
                'gamma': ['scale', 'auto', 0.1, 0.01],
                'kernel': ['rbf']
            }

scaler_std = StandardScaler()
scaler_std.fit_transform(X_train)
scaler_std.transform(X_test)

grid_search_svm = GridSearchCV(svm, param_grid=svm_grid, scoring = 'accuracy', cv =cv, verbose = 1)
grid_search_svm.fit(X_train, y_train)

print("Best score", grid_search_svm.best_score_)
y_pred_svm = grid_search_svm.best_estimator_.predict(X_test)

print(classification_report(y_pred_svm, y_test))